In [20]:
import pandas as pd
import numpy as np
import random
df=pd.read_csv('tiempos_por_tecla.csv')
df = df.drop(df[df.modo != 1].index)
df
prueba = ['usuario','practica']

In [21]:
df.head(10)
# sacar promedio de mantener la letra pulsada

,tiempo_id,usuario,practica,tecla,up,down,anterior,anterior_down,modo
0,1,1,1,69,43,0,0,0,1
1,2,1,1,83,237,157,69,0,1
2,3,1,1,84,458,330,83,157,1
3,4,1,1,65,571,426,84,330,1
4,5,1,1,32,650,572,65,426,1
5,6,1,1,69,778,714,32,572,1
6,7,1,1,83,970,858,69,714,1
7,8,1,1,32,1066,986,83,858,1
8,9,1,1,85,1226,1146,32,986,1
9,10,1,1,78,1418,1338,85,1146,1


In [22]:
df['tiempo'] = df.down - df.anterior_down
df['anterior_up'] = df.up.shift(periods = 1)
df['anterior_up'] = np.where(df['down'] == 0, 0, df['anterior_up'])
df['solapa'] = np.where((df.anterior_up - df.down) > 0, 1, 0)
df['tecla'] = np.where(df.tecla == 32, 95, df.tecla)
df['anterior'] = np.where(df.anterior == 32, 95, df.anterior)
df['tecla'] = df['tecla'].apply(chr)
df['anterior'] = df['anterior'].apply(chr)
df['par'] = df.anterior + df.tecla

pt = df.groupby(by=['usuario','practica','par'])['tiempo'].median()
pt = pt.reset_index()
pt = pt.rename(columns={'tiempo':'mean'})
pa = df.groupby(by=['usuario','practica'])['solapa'].sum()
pa = pa.reset_index()
pl = df.groupby(by=['usuario','practica','par'])['tiempo'].median()
pl = pl.reset_index()
pstd = df.groupby(by=['usuario','practica','par'])['tiempo'].std()
pstd = pl.reset_index()
pstd = pstd.rename(columns={'tiempo':'std'})
pstd

,index,usuario,practica,par,std
0,0,1,1, E,0.0
1,1,1,1,",_",895.5
2,2,1,1,.0,224.0
3,3,1,1,.1,320.0
4,4,1,1,.5,279.5
...,...,...,...,...,...
72081,72081,93,30,_Q,104.0
72082,72082,93,30,_S,135.0
72083,72083,93,30,_T,128.0
72084,72084,93,30,_U,156.0


In [23]:
pt

,usuario,practica,par,mean
0,1,1, E,0.0
1,1,1,",_",895.5
2,1,1,.0,224.0
3,1,1,.1,320.0
4,1,1,.5,279.5
...,...,...,...,...
72081,93,30,_Q,104.0
72082,93,30,_S,135.0
72083,93,30,_T,128.0
72084,93,30,_U,156.0


In [24]:
pl = pl.pivot(index=prueba, columns='par', values='tiempo')
pl = pl.reset_index()
pstd = pstd.pivot(index=prueba, columns='par', values='std')
pstd = pstd.reset_index()
pstd

par,usuario,practica, E,",_",.0,.1,.5,.E,04,15,...,_I,_J,_L,_O,_P,_Q,_S,_T,_U,_V
0,1,1,0.0,895.5,224.0,320.0,279.5,1185.0,129.0,128.0,...,168.0,290.0,145.0,211.0,126.0,175.0,206.0,848.0,184.0,111.5
1,1,2,0.0,200.0,353.0,255.0,264.5,1056.0,174.0,146.0,...,168.0,464.0,144.0,224.0,128.0,192.0,160.0,430.0,678.5,127.5
2,1,3,0.0,207.5,241.0,256.0,313.0,974.0,127.0,129.0,...,154.0,161.0,446.0,238.0,161.0,224.0,192.0,256.0,192.0,145.0
3,1,4,0.0,175.5,224.0,256.0,271.5,1281.0,112.0,143.0,...,328.0,176.0,150.0,320.0,129.0,192.0,159.0,288.0,175.5,184.0
4,1,5,0.0,176.0,609.0,223.0,464.5,1104.0,176.0,191.0,...,177.0,159.0,176.0,239.0,143.0,176.0,194.0,240.0,216.5,512.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
537,93,26,0.0,128.0,177.0,233.0,188.5,1088.0,223.0,71.0,...,200.0,152.0,176.0,144.0,90.0,154.0,192.0,112.0,123.5,131.5
538,93,27,0.0,172.5,224.0,222.0,184.0,664.0,200.0,121.0,...,160.0,200.0,128.0,104.0,120.0,152.0,775.0,88.0,163.5,211.5
539,93,28,0.0,100.5,176.0,191.0,220.0,696.0,216.0,105.0,...,412.0,192.0,127.0,89.0,80.0,129.0,143.0,80.0,132.5,159.5
540,93,29,0.0,116.5,175.0,200.0,200.0,872.0,226.0,96.0,...,136.0,152.0,168.0,176.0,80.0,144.0,408.0,96.0,140.5,144.0


In [25]:
pp = df.merge(pl, on=prueba, how='outer')
pp = pp.merge(pstd, on=prueba, how='outer',suffixes=('', 'm'))
#pp['lec'] = np.where((pp['mean'] + pp['std']) < pp['tiempo'], 1, 0)
pp

,tiempo_id,usuario,practica,tecla,up,down,anterior,anterior_down,modo,tiempo,...,_Im,_Jm,_Lm,_Om,_Pm,_Qm,_Sm,_Tm,_Um,_Vm
0,1,1,1,E,43,0, ,0,1,0,...,168.0,290.0,145.0,211.0,126.0,175.0,206.0,848.0,184.0,111.5
1,2,1,1,S,237,157,E,0,1,157,...,168.0,290.0,145.0,211.0,126.0,175.0,206.0,848.0,184.0,111.5
2,3,1,1,T,458,330,S,157,1,173,...,168.0,290.0,145.0,211.0,126.0,175.0,206.0,848.0,184.0,111.5
3,4,1,1,A,571,426,T,330,1,96,...,168.0,290.0,145.0,211.0,126.0,175.0,206.0,848.0,184.0,111.5
4,5,1,1,_,650,572,A,426,1,146,...,168.0,290.0,145.0,211.0,126.0,175.0,206.0,848.0,184.0,111.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136037,228727,64,39,A,52693,52574,U,52468,1,106,...,211.5,193.0,112.0,184.0,103.0,119.0,98.0,184.0,167.0,135.5
136038,228728,64,39,R,52772,52670,A,52574,1,96,...,211.5,193.0,112.0,184.0,103.0,119.0,98.0,184.0,167.0,135.5
136039,228729,64,39,I,52860,52756,R,52670,1,86,...,211.5,193.0,112.0,184.0,103.0,119.0,98.0,184.0,167.0,135.5
136040,228730,64,39,O,52925,52828,I,52756,1,72,...,211.5,193.0,112.0,184.0,103.0,119.0,98.0,184.0,167.0,135.5


In [26]:
lis = df.par.unique()
for k in lis:
    col = str(k) + 'out'
    pmean = str(k)
    pstd = str(k) + 'm'
    pp[col] = np.where((pp[pmean] + pp[pstd]) < pp['tiempo'], 1, 0)
pp

/tmp/ipykernel_23235/3283557062.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  pp[col] = np.where((pp[pmean] + pp[pstd]) < pp['tiempo'], 1, 0)


,tiempo_id,usuario,practica,tecla,up,down,anterior,anterior_down,modo,tiempo,...,ALout,LIout,IDout,DAout,L_out,USout,SUout,UAout,IOout,O.out
0,1,1,1,E,43,0, ,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,2,1,1,S,237,157,E,0,1,157,...,0,0,0,0,0,0,0,0,1,0
2,3,1,1,T,458,330,S,157,1,173,...,0,0,0,0,0,0,1,0,1,0
3,4,1,1,A,571,426,T,330,1,96,...,0,0,0,0,0,0,0,0,1,0
4,5,1,1,_,650,572,A,426,1,146,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136037,228727,64,39,A,52693,52574,U,52468,1,106,...,0,0,0,0,0,0,0,0,0,0
136038,228728,64,39,R,52772,52670,A,52574,1,96,...,0,0,0,0,0,0,0,0,0,0
136039,228729,64,39,I,52860,52756,R,52670,1,86,...,0,0,0,0,0,0,0,0,0,0
136040,228730,64,39,O,52925,52828,I,52756,1,72,...,0,0,0,0,0,0,0,0,0,0


In [27]:
#pe = pp.groupby(by=['usuario','practica','modo'])['lec'].sum()
#pe = pe.reset_index()
#pe

In [28]:
#pc = pp.groupby(by=['usuario','practica','modo'])['lec'].count()
#pc = pc.reset_index()
#pc = pc.rename(columns={'lec': 'cant'})
#pc

In [29]:
#pe = pe.merge(pc, on=['usuario','practica','modo'])
#pe

In [30]:
#pe['mean_error'] = pe['cant'] / pe['lec']
#pe

In [31]:
pa

,usuario,practica,solapa
0,1,1,62
1,1,2,57
2,1,3,55
3,1,4,62
4,1,5,60
...,...,...,...
537,93,26,97
538,93,27,113
539,93,28,111
540,93,29,120


In [32]:
final = pp.merge(pa, on=prueba)
#final['prom_solapa'] = final['cant'] / final['solapa'] 
#final = final.merge(pl, on=prueba, how='outer')
final

,tiempo_id,usuario,practica,tecla,up,down,anterior,anterior_down,modo,tiempo,...,LIout,IDout,DAout,L_out,USout,SUout,UAout,IOout,O.out,solapa_y
0,1,1,1,E,43,0, ,0,1,0,...,0,0,0,0,0,0,0,0,0,62
1,2,1,1,S,237,157,E,0,1,157,...,0,0,0,0,0,0,0,1,0,62
2,3,1,1,T,458,330,S,157,1,173,...,0,0,0,0,0,1,0,1,0,62
3,4,1,1,A,571,426,T,330,1,96,...,0,0,0,0,0,0,0,1,0,62
4,5,1,1,_,650,572,A,426,1,146,...,0,0,0,0,0,0,0,1,0,62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136037,228727,64,39,A,52693,52574,U,52468,1,106,...,0,0,0,0,0,0,0,0,0,39
136038,228728,64,39,R,52772,52670,A,52574,1,96,...,0,0,0,0,0,0,0,0,0,39
136039,228729,64,39,I,52860,52756,R,52670,1,86,...,0,0,0,0,0,0,0,0,0,39
136040,228730,64,39,O,52925,52828,I,52756,1,72,...,0,0,0,0,0,0,0,0,0,39


In [33]:
final = final.drop(columns=['anterior','tecla','par','tiempo_id'])
final

,usuario,practica,up,down,anterior_down,modo,tiempo,anterior_up,solapa_x, E,...,LIout,IDout,DAout,L_out,USout,SUout,UAout,IOout,O.out,solapa_y
0,1,1,43,0,0,1,0,0.0,0,0.0,...,0,0,0,0,0,0,0,0,0,62
1,1,1,237,157,0,1,157,43.0,0,0.0,...,0,0,0,0,0,0,0,1,0,62
2,1,1,458,330,157,1,173,237.0,0,0.0,...,0,0,0,0,0,1,0,1,0,62
3,1,1,571,426,330,1,96,458.0,1,0.0,...,0,0,0,0,0,0,0,1,0,62
4,1,1,650,572,426,1,146,571.0,0,0.0,...,0,0,0,0,0,0,0,1,0,62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136037,64,39,52693,52574,52468,1,106,52573.0,0,0.0,...,0,0,0,0,0,0,0,0,0,39
136038,64,39,52772,52670,52574,1,96,52693.0,1,0.0,...,0,0,0,0,0,0,0,0,0,39
136039,64,39,52860,52756,52670,1,86,52772.0,1,0.0,...,0,0,0,0,0,0,0,0,0,39
136040,64,39,52925,52828,52756,1,72,52860.0,1,0.0,...,0,0,0,0,0,0,0,0,0,39


In [34]:
columns = []
columns = list(final.columns.values)
pro = final.groupby(by=['usuario','practica'])['modo'].mean()
pro = pro.reset_index()
for i in columns:
    if ((i!='usuario')&(i!='practica')):
        p = final.groupby(by=['usuario','practica'])[i].mean()
        p = p.reset_index()
        pro = pro.merge(p, on=prueba)

pro

,usuario,practica,modo_x,up,down,anterior_down,modo_y,tiempo,anterior_up,solapa_x,...,LIout,IDout,DAout,L_out,USout,SUout,UAout,IOout,O.out,solapa_y
0,1,1,1.0,26180.808765,26089.019920,25880.406375,1.0,208.613546,25972.195219,0.247012,...,0.167331,0.235060,0.183267,0.254980,0.127490,0.394422,0.163347,0.964143,0.039841,62.0
1,1,2,1.0,25041.079681,24948.258964,24756.505976,1.0,191.752988,24849.326693,0.227092,...,0.247012,0.151394,0.247012,0.183267,0.015936,0.091633,0.247012,0.430279,0.035857,57.0
2,1,3,1.0,24609.171315,24517.215139,24327.904382,1.0,189.310757,24419.860558,0.219124,...,0.187251,0.187251,0.278884,0.087649,0.167331,0.207171,0.414343,0.617530,0.027888,55.0
3,1,4,1.0,23621.653386,23526.382470,23340.322709,1.0,186.059761,23435.593625,0.247012,...,0.103586,0.155378,0.191235,0.179283,0.055777,0.565737,0.250996,0.565737,0.047809,62.0
4,1,5,1.0,22721.442231,22628.187251,22441.601594,1.0,186.585657,22534.856574,0.239044,...,0.187251,0.235060,0.482072,0.051793,0.047809,0.438247,0.179283,0.848606,0.047809,60.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
537,93,26,1.0,19649.159363,19539.780876,19381.565737,1.0,158.215139,19490.944223,0.386454,...,0.342629,0.414343,0.099602,0.071713,0.099602,0.099602,0.207171,0.342629,0.019920,97.0
538,93,27,1.0,18386.776892,18276.760956,18132.629482,1.0,144.131474,18242.645418,0.450199,...,0.239044,0.239044,0.079681,0.239044,0.155378,0.079681,0.187251,0.239044,0.023904,113.0
539,93,28,1.0,18069.764940,17962.231076,17817.661355,1.0,144.569721,17925.195219,0.442231,...,0.498008,0.067729,0.055777,0.051793,0.191235,0.067729,0.219124,0.378486,0.027888,111.0
540,93,29,1.0,17688.075697,17576.988048,17432.031873,1.0,144.956175,17543.119522,0.478088,...,0.330677,0.290837,0.119522,0.043825,0.063745,0.095618,0.119522,0.159363,0.023904,120.0


In [35]:
prod = pro
#prod = prod.where( prod >= 0 , prod['mean'], axis=0)
prod = prod.where( prod != np.inf , 0, axis=0)
prod

,usuario,practica,modo_x,up,down,anterior_down,modo_y,tiempo,anterior_up,solapa_x,...,LIout,IDout,DAout,L_out,USout,SUout,UAout,IOout,O.out,solapa_y
0,1,1,1.0,26180.808765,26089.019920,25880.406375,1.0,208.613546,25972.195219,0.247012,...,0.167331,0.235060,0.183267,0.254980,0.127490,0.394422,0.163347,0.964143,0.039841,62.0
1,1,2,1.0,25041.079681,24948.258964,24756.505976,1.0,191.752988,24849.326693,0.227092,...,0.247012,0.151394,0.247012,0.183267,0.015936,0.091633,0.247012,0.430279,0.035857,57.0
2,1,3,1.0,24609.171315,24517.215139,24327.904382,1.0,189.310757,24419.860558,0.219124,...,0.187251,0.187251,0.278884,0.087649,0.167331,0.207171,0.414343,0.617530,0.027888,55.0
3,1,4,1.0,23621.653386,23526.382470,23340.322709,1.0,186.059761,23435.593625,0.247012,...,0.103586,0.155378,0.191235,0.179283,0.055777,0.565737,0.250996,0.565737,0.047809,62.0
4,1,5,1.0,22721.442231,22628.187251,22441.601594,1.0,186.585657,22534.856574,0.239044,...,0.187251,0.235060,0.482072,0.051793,0.047809,0.438247,0.179283,0.848606,0.047809,60.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
537,93,26,1.0,19649.159363,19539.780876,19381.565737,1.0,158.215139,19490.944223,0.386454,...,0.342629,0.414343,0.099602,0.071713,0.099602,0.099602,0.207171,0.342629,0.019920,97.0
538,93,27,1.0,18386.776892,18276.760956,18132.629482,1.0,144.131474,18242.645418,0.450199,...,0.239044,0.239044,0.079681,0.239044,0.155378,0.079681,0.187251,0.239044,0.023904,113.0
539,93,28,1.0,18069.764940,17962.231076,17817.661355,1.0,144.569721,17925.195219,0.442231,...,0.498008,0.067729,0.055777,0.051793,0.191235,0.067729,0.219124,0.378486,0.027888,111.0
540,93,29,1.0,17688.075697,17576.988048,17432.031873,1.0,144.956175,17543.119522,0.478088,...,0.330677,0.290837,0.119522,0.043825,0.063745,0.095618,0.119522,0.159363,0.023904,120.0


In [36]:
#prod['usuario'] = np.where(prod['usuario']==1, True, False)
prod

,usuario,practica,modo_x,up,down,anterior_down,modo_y,tiempo,anterior_up,solapa_x,...,LIout,IDout,DAout,L_out,USout,SUout,UAout,IOout,O.out,solapa_y
0,1,1,1.0,26180.808765,26089.019920,25880.406375,1.0,208.613546,25972.195219,0.247012,...,0.167331,0.235060,0.183267,0.254980,0.127490,0.394422,0.163347,0.964143,0.039841,62.0
1,1,2,1.0,25041.079681,24948.258964,24756.505976,1.0,191.752988,24849.326693,0.227092,...,0.247012,0.151394,0.247012,0.183267,0.015936,0.091633,0.247012,0.430279,0.035857,57.0
2,1,3,1.0,24609.171315,24517.215139,24327.904382,1.0,189.310757,24419.860558,0.219124,...,0.187251,0.187251,0.278884,0.087649,0.167331,0.207171,0.414343,0.617530,0.027888,55.0
3,1,4,1.0,23621.653386,23526.382470,23340.322709,1.0,186.059761,23435.593625,0.247012,...,0.103586,0.155378,0.191235,0.179283,0.055777,0.565737,0.250996,0.565737,0.047809,62.0
4,1,5,1.0,22721.442231,22628.187251,22441.601594,1.0,186.585657,22534.856574,0.239044,...,0.187251,0.235060,0.482072,0.051793,0.047809,0.438247,0.179283,0.848606,0.047809,60.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
537,93,26,1.0,19649.159363,19539.780876,19381.565737,1.0,158.215139,19490.944223,0.386454,...,0.342629,0.414343,0.099602,0.071713,0.099602,0.099602,0.207171,0.342629,0.019920,97.0
538,93,27,1.0,18386.776892,18276.760956,18132.629482,1.0,144.131474,18242.645418,0.450199,...,0.239044,0.239044,0.079681,0.239044,0.155378,0.079681,0.187251,0.239044,0.023904,113.0
539,93,28,1.0,18069.764940,17962.231076,17817.661355,1.0,144.569721,17925.195219,0.442231,...,0.498008,0.067729,0.055777,0.051793,0.191235,0.067729,0.219124,0.378486,0.027888,111.0
540,93,29,1.0,17688.075697,17576.988048,17432.031873,1.0,144.956175,17543.119522,0.478088,...,0.330677,0.290837,0.119522,0.043825,0.063745,0.095618,0.119522,0.159363,0.023904,120.0


In [37]:
from sklearn.model_selection import train_test_split
from sklearn import linear_model        
from sklearn import neighbors           
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn import linear_model
X=prod.drop(axis=1,columns='usuario')  
y=prod['usuario']      
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=123)
X_val_train,X_val_test,y_val_train,y_val_test=train_test_split(X_train,y_train,test_size=0.20,random_state=123)

k_valores=np.arange(1,51,1)   
k_max=0
MRSEkNN_max=0
for k in k_valores:
    modelokNN = neighbors.KNeighborsClassifier(n_neighbors=k,weights='uniform')
    modelokNN.fit(X_val_train, y_val_train)
    y_val_pred=modelokNN.predict(X_val_test)
    MRSEkNN=accuracy_score(y_val_test, y_val_pred)
    
    if  MRSEkNN>MRSEkNN_max:
        k_max=k
        MRSEkNN_max=MRSEkNN
    
print('Knn con peso uniforme')
print('Valor de k: ', k_max)
print("Mejor MRSE: ", round(MRSEkNN_max,3))

k_valores=np.arange(1,51,1)   
k_max=0
MRSEkNN_max=0
for k in k_valores:
    modelokNN = neighbors.KNeighborsClassifier(n_neighbors=k,weights='distance')
    modelokNN.fit(X_val_train, y_val_train)
    y_val_pred=modelokNN.predict(X_val_test)
    MRSEkNN=accuracy_score(y_val_test, y_val_pred)
    
    if  MRSEkNN>MRSEkNN_max:
        k_max=k
        MRSEkNN_max=MRSEkNN

print('KNn con un peso en distancia')    
print('Valor de k: ', k_max)
print("Mejor MRSE: ", round(MRSEkNN_max,3))



profundidades=np.arange(1,31,1)
p_max=0               
MRSEkNNT_max=0

print("Evaluación modelo Árbol de Decisión en el Validation")

for p in profundidades: 
    modeloT=DecisionTreeClassifier(max_depth=p, random_state=123)        
    modeloT.fit(X_val_train, y_val_train)             
    y_val_pred = modeloT.predict(X_val_test)           
    MRSEkNNT=accuracy_score(y_val_test, y_val_pred)
    
    if  MRSEkNNT>MRSEkNNT_max:
        p_max=p
        MRSEkNNT_max=MRSEkNNT

print('Valor de k: ', p_max)
print("Mejor MRSE: ", round(MRSEkNNT_max,3))

modelo=linear_model.LogisticRegression(max_iter=1000,penalty='none',fit_intercept=True, random_state=123)
modelo.fit(X_val_train,y_val_train)
y_val_pred=modelo.predict(X_val_test)
AC=accuracy_score(y_val_test, y_val_pred)

print("Regrecion: ")
print("Accuracy: ", round(AC,3))



Knn con peso uniforme
Valor de k:  14
Mejor MRSE:  0.368
KNn con un peso en distancia
Valor de k:  17
Mejor MRSE:  0.379
Evaluación modelo Árbol de Decisión en el Validation
Valor de k:  16
Mejor MRSE:  0.747
Regrecion: 
Accuracy:  0.678


In [38]:
res = [random.randrange(0, len(prod), 1) for i in range(20)]
res 
for k in res:
    dt = prod.copy(deep=True)
    dt['usuario'] = np.where( dt['usuario']==k , 1 , 0)
    X=dt.drop(axis=1,columns='usuario')  
    y=dt['usuario']      
    X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=123)
    X_val_train,X_val_test,y_val_train,y_val_test=train_test_split(X_train,y_train,test_size=0.20,random_state=123)
    modeloT=DecisionTreeClassifier(max_depth=16, random_state=123)        
    modeloT.fit(X_val_train, y_val_train)             
    y_val_pred = modeloT.predict(X_val_test)           
    MRSEkNNT=accuracy_score(y_val_test, y_val_pred)
    #print(str(p) + "  " + str(k))
    print(MRSEkNNT)
    

1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0


In [39]:
def perf_measure(y_actual, y_hat):
    TP = 0
    FP = 0
    TN = 0
    FN = 0

    for i in range(len(y_hat)): 
        if y_actual[i]==y_hat[i]==1:
           TP += 1
        if y_hat[i]==1 and y_actual[i]!=y_hat[i]:
           FP += 1
        if y_actual[i]==y_hat[i]==0:
           TN += 1
        if y_hat[i]==0 and y_actual[i]!=y_hat[i]:
           FN += 1

    return(TP, FP, TN, FN)

In [40]:
res = prod['usuario'].unique()
res 
top = 0
ton = 0
fn = 0
fp = 0
tn = 0
tp = 0
for k in res:
    dt = prod.copy(deep=True)
    dt['usuario'] = np.where( dt['usuario']==k , 1 , 0)
    X=dt.drop(axis=1,columns='usuario')  
    y=dt['usuario']      
    X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=123)
    X_val_train,X_val_test,y_val_train,y_val_test=train_test_split(X_train,y_train,test_size=0.20,random_state=123)
    modeloT=DecisionTreeClassifier(max_depth=16, random_state=123)        
    modeloT.fit(X_val_train, y_val_train)             
    y_val_pred = modeloT.predict(X_val_test)           
    MRSEkNNT=accuracy_score(y_val_test, y_val_pred)
    print(MRSEkNNT)
    y_val_pred = y_val_pred.tolist()
    y_val_test = y_val_test.tolist()
    TP, FP, TN, FN = perf_measure(y_val_pred, y_val_test)
    fn = fn + FN
    fp = fp + FP
    tn = tn + TN
    tp = tp + TP
t = tp + fp +tn + fn

print("total: " + str(t) + " Falso positivo: " + str(fp) + " Falso negativo: " + str(fn) + " Acierto positivo: " + str(tp) + " Acierto negativo: " + str(tn))
print("total: " + str(t) + " Falso positivo: " + str((fp/t)*100) + " Falso negativo: " + str((fn/t)*100) + " Acierto positivo: " + str((tp/t)*100) + " Acierto negativo: " + str((tn/t)*100))

0.9310344827586207
0.9885057471264368
0.9540229885057471
0.9655172413793104
0.9425287356321839
0.9655172413793104
0.9655172413793104
0.9540229885057471
0.9885057471264368
0.9540229885057471
0.9655172413793104
0.9885057471264368
0.9885057471264368
0.9310344827586207
0.9540229885057471
1.0
1.0
1.0
0.9540229885057471
1.0
0.9885057471264368
0.9540229885057471
0.9885057471264368
0.9770114942528736
0.9770114942528736
0.9885057471264368
0.9885057471264368
1.0
total: 2436 Falso positivo: 31 Falso negativo: 34 Acierto positivo: 56 Acierto negativo: 2315
total: 2436 Falso positivo: 1.2725779967159279 Falso negativo: 1.3957307060755337 Acierto positivo: 2.2988505747126435 Acierto negativo: 95.03284072249589


In [41]:
print((fp/t)*100)

1.2725779967159279
